# Convert Runtime/(Visited Subgroups) Speedups from pandas-compatible .csv File to LaTeX Code

## Default Values for Papermill Parameters

In [ ]:
PARAM_RUNTIME_SPEEDUPS_PATH = "../outputs/speedups_statistics_merged_result_set.csv"
PARAM_OUT_FILENAME_SUFFIX = "speedups_table.tex"

## Import and Set Parameters

In [ ]:
from subroc import util

import pandas as pd
import os

# fill environment variables into params
PARAM_RUNTIME_SPEEDUPS_PATH = util.prepend_experiment_output_path(PARAM_RUNTIME_SPEEDUPS_PATH)

# get environment variables
STAGE_OUTPUT_PATH = os.environ.get("STAGE_OUTPUT_PATH", "../outputs")

## Read the Speedups

In [ ]:
speedups_df = pd.read_csv(PARAM_RUNTIME_SPEEDUPS_PATH)

print(speedups_df)

## Prepare the Runtime Table Entries

In [ ]:
qf_names = {
    "average_ranking_loss": r"$\varphi_{Adult}^{rasl}$",
    "roc_auc_score": r"$\varphi_{Adult}^{ROCAUC}$",
    "prc_auc_score": r"$\varphi_{Adult}^{PRAUC}$",
}

df_groupby = speedups_df.groupby("qf_name", as_index=False)
latex_rows = []

for key in df_groupby.groups.keys():
    group = df_groupby.get_group(key)

    qf_name = qf_names[key]
    latex_rows.append([qf_name, *[f"{time_speedup:.4f}" for time_speedup in group["time_speedup"]]])

runtime_latex_df = pd.DataFrame(latex_rows, columns=["Interestingness Measure", *[r"$\theta_{dc}=" + str(i+1) + "$" for i in range(len(latex_rows[0])-1)]])

print(runtime_latex_df)

## Write the Runtime Speedups Table to .tex

In [ ]:
runtime_latex_df.to_latex(f"{STAGE_OUTPUT_PATH}/runtime_{PARAM_OUT_FILENAME_SUFFIX}", float_format="%.4f", index=False)

## Prepare the Visited Subgroups Speedups Table Entries

In [ ]:
latex_rows = []

for key in df_groupby.groups.keys():
    group = df_groupby.get_group(key)

    qf_name = qf_names[key]
    latex_rows.append([qf_name, *[fr"{100*num_visited_subgroups:.2f}\%" for num_visited_subgroups in group["num_visited_subgroups_speedup"]]])

num_visited_subgroups_latex_df = pd.DataFrame(latex_rows, columns=["Interestingness Measure", *[r"$\theta_{dc}=" + str(i+1) + "$" for i in range(len(latex_rows[0])-1)]])

print(num_visited_subgroups_latex_df)

## Write the Visited Subgroups Speedups Table to .tex

In [ ]:
num_visited_subgroups_latex_df.to_latex(f"{STAGE_OUTPUT_PATH}/num_visited_subgroups_{PARAM_OUT_FILENAME_SUFFIX}", float_format="%.4f", index=False)